In [3]:
from bs4 import BeautifulSoup as bs
import requests as r
import pandas as pd
import datetime as dt

metals = ['XAU','XAG']
currencies = ['USD','GBP']

base_url = 'https://www.exchangerates.org.uk/commodities/'
base = r.get(base_url)
soup = bs(base.text, features="html.parser")

In [4]:
df_data = []
for metal in metals:
    for currency in currencies:
        url = f'{base_url}{metal}-{currency}-history.html'
        base = r.get(url)
        soup = bs(base.text, features="html.parser")
        tbl = soup.find('table')
        Last_updated = tbl.find('i').text
        tbl_rows = tbl.find_all('tr')[2:]
        for row in tbl_rows:
            data = row.find_all('td')
            rowdate = dt.datetime.strptime(data[0].text,'%A %d %B %Y')
            rowamt = data[2].text
            rowdata = (rowdate,rowamt,metal,currency)
            df_data.append(rowdata)


In [84]:
df = pd.DataFrame(df_data)
df.columns = ['date','Amount','Comod','Cnty']

In [72]:
df.set_index(['Comod','Cnty','date'], inplace=True)
df

Amount
Comod Cnty date                  
XAU   USD  2020-03-01  1583.93500
           2020-02-29  1584.81500
           2020-02-28  1584.81500
           2020-02-27  1645.59000
           2020-02-26  1643.09000
...                           ...
XAG   GBP  2010-11-17    16.18380
           2010-11-16    15.99962
           2010-11-15    15.90864
           2010-11-14    16.13828
           1970-01-01    23.27959

[13592 rows x 1 columns]

In [73]:
df.sort_index(inplace=True)

In [74]:
df['3MA'] = df.Amount.rolling(3).mean()
#out = df.groupby(level=['Comod','Cnty']).apply(lambda x: x.rolling(3).mean())

In [75]:
#out.to_csv('out.csv')

In [83]:
df


Amount
Comod Cnty date                  
XAG   GBP  1970-01-01    23.27959
           2010-11-14    16.13828
           2010-11-15    15.90864
           2010-11-16    15.99962
           2010-11-17    16.18380
...                           ...
XAU   USD  2020-02-26  1643.09000
           2020-02-27  1645.59000
           2020-02-28  1584.81500
           2020-02-29  1584.81500
           2020-03-01  1583.93500

[13592 rows x 1 columns]

In [95]:
df.sort_values('date', inplace=True)
df

,date,Amount,Comod,Cnty
6795,1970-01-01,891.42841,XAU,GBP
10193,1970-01-01,37.34500,XAG,USD
3397,1970-01-01,1430.02500,XAU,USD
13591,1970-01-01,23.27959,XAG,GBP
3396,2010-11-14,1368.72500,XAU,USD
...,...,...,...,...
1,2020-02-29,1584.81500,XAU,USD
6796,2020-03-01,16.55300,XAG,USD
3398,2020-03-01,1237.08679,XAU,GBP
10194,2020-03-01,12.92824,XAG,GBP


In [107]:
df['3MA'] = df.groupby(['Comod','Cnty']).rolling(3)['Amount'].mean().reset_index(0,drop=True)



TypeError: incompatible index of inserted column with frame index